In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
from build_dataset import *
import json
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))
from utils import *

In [3]:
ds = datasets.load_dataset("franlucc/stenotype-eval-dataset", split="train")

In [4]:
f_ds = filter_types_with_idx(ds, QUERY_FUNC_TYPES)
f_ds

Dataset({
    features: ['hexsha', 'size', 'ext', 'lang', 'max_stars_repo_path', 'max_stars_repo_name', 'max_stars_repo_head_hexsha', 'max_stars_repo_licenses', 'max_stars_count', 'max_stars_repo_stars_event_min_datetime', 'max_stars_repo_stars_event_max_datetime', 'max_issues_repo_path', 'max_issues_repo_name', 'max_issues_repo_head_hexsha', 'max_issues_repo_licenses', 'max_issues_count', 'max_issues_repo_issues_event_min_datetime', 'max_issues_repo_issues_event_max_datetime', 'max_forks_repo_path', 'max_forks_repo_name', 'max_forks_repo_head_hexsha', 'max_forks_repo_licenses', 'max_forks_count', 'max_forks_repo_forks_event_min_datetime', 'max_forks_repo_forks_event_max_datetime', 'content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'annotation_sites', 'type_definitions', 'loc', 'functions', 'loc_per_function', 'estimated_tokens', 'fim_program', 'fim_type'],
    num_rows: 5370
})

In [5]:
i = 0
print(f_ds[i]["fim_program"])
print(f_ds[i]["fim_type"])

/*
 * @Author: your name
 * @Date: 2022-04-24 10:11:23
 * @LastEditTime: 2022-04-24 16:14:52
 * @LastEditors: Please set LastEditors
 * @Description: 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
 * @FilePath: /tinkerbell-ui-react/src/packages/DaysPicker/date/interface.ts
 */
//  props 天
export interface dayProps {
    year: number;
    month: number;
    active: number;
    change: Function;
    visible: boolean;
    monthRef: any;
    show: boolean;
    limit: boolean;
}
//  props 月
export interface monthProps {
    visible: boolean;
    active: number;
    change: Function;
    year: number;
    plateChange: Function;
    limit: boolean;
}
//  props 年
export interface yearProps {
    visible: boolean;
    active: number;
    change: Function;
    plateChange: Function;
    limit: boolean;
}

// pageday-item type
export type dayItem = {
    label: string;
    value: number;
    disabled: boolean; //禁用
    prev: boolean; // 是否属于上一月份
    

In [7]:
f_ds.push_to_hub("franlucc/stenotype-eval-dataset-func-type-stripped-v3")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
f_ds_v0 = datasets.load_dataset("franlucc/stenotype-eval-func-type-stripped-v1", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/336 [00:00<?, ? examples/s]

In [9]:
count = 0
diffs = []
lens = []
for i in range(len(f_ds)):
    a =f_ds[i]["content_type_removed"]
    b=f_ds_v0[i]["content_type_removed"]
    if a != b:
        count += 1
        diffs.append((i,a, b))
        lens.append(len(a) - len(b))
        
count

0